In [6]:
""""
Figaro times collection times script. 
"""
import numpy as np
import pandas as pd
import os
import regex as re
from openpyxl import load_workbook, Workbook
from openpyxl.utils.cell import get_column_letter

exp_names = ["svd_mkl_dc", "svd_figaro"]
exp_comp = {"svd_mkl_dc"}
exp_fig = "svd_figaro"
#/local/scratch/Figaro/tests_path/figaro-code/dumps/figaro/svd/divide_and_conquer/householder/sing_vals_100/s_onlyt/hread48/DBRetailerPK1C100/ItemRoot/R.csv
exp_paths = {"svd_mkl_dc": "/local/scratch/Figaro/tests_path/figaro-code/dumps/decomp_alg/svd/divide_and_conquer/sing_vals_100/col_major/s_only", "svd_figaro": "/local/scratch/Figaro/tests_path/figaro-code/dumps/figaro/svd/divide_and_conquer/householder/sing_vals_100/s_onlyt/hread48"}
#db_names = ["DBRetailer", "DBFavorita", "DBYelp"]
db_names = ["DBRetailer", "DBFavorita", "DBYelp"]
#join_orders = {"DBRetailer": "LocationRoot", "DBFavorita": "StoresRoot", "DBYelp": "BusinessRoot"}
join_orders = {"DBRetailer": "ItemRoot", "DBFavorita": "ItemsRoot", "DBYelp": "HoursRoot"}
start_per = 10
end_per = 100
per_inc = 10
ohe = False

csv_name = "R.csv"
numeric_const_pattern = r"Relative error is: ([+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?)"
rx = re.compile(numeric_const_pattern)

df_measurement_exps = {}

df_measurement = pd.DataFrame(columns=db_names)

np.set_printoptions(precision=20)
for db_name in db_names:
    join_order = join_orders[db_name]

    db_name_per = "{}{}".format(db_name, "PK1C100")
    

    exp_path = exp_paths["svd_figaro"]
    path_csv = os.path.join(exp_path, db_name_per,
    join_order, csv_name)
    print(path_csv)
    np_figaro = pd.read_csv(path_csv, header=None).astype(float).to_numpy()
    
    exp_path = exp_paths["svd_mkl_dc"]
    path_csv = os.path.join(exp_path, db_name_per,
    join_order, csv_name)
    np_mkl = pd.read_csv(path_csv, header=None).astype(float).to_numpy()
    
    #print(np_figaro)
    #print(np_mkl)
    (len, _) = np.shape(np_mkl)
    print(len)
    dist = per_inc / 100.0 * len
    curDist = 0.0
    print(dist)
    for db_idx, percent in enumerate(range(start_per, end_per + 1, per_inc)):
        per_val = (int)(percent / 100.0 * np_figaro.shape[0])
        per_prev_val = (int)((percent - 10) / 100.0 * np_figaro.shape[0]) + 1
        print("percent", per_val)
        np_fig_per = np_figaro[per_prev_val:per_val]
        print(np_fig_per)
        print(np_mkl_per)
        np_mkl_per = np_mkl[per_prev_val:per_val]
        np_diff = np_fig_per - np_mkl_per
        
        

        abs_err = np.linalg.norm(np_diff, ord='fro')
        abs_err_comp  = np.linalg.norm(np_mkl_per, ord='fro')
        print("percent", percent, per_prev_val, next_val)
        print(np_diff)
        #print("figaro", np_fig_per, np_fig_per.shape)
        #print("original", np_mkl_per, np_mkl_per.shape)
        #if (percent == 100):
            #print(np_mkl_per)
            #print(np_figaro)
        relative_frob_norm = abs_err / abs_err_comp

        df_measurement.at[percent, db_name] = relative_frob_norm
        curDist += dist
        
dir_out = "ohe" if ohe else "non-ohe"
os.makedirs(dir_out, exist_ok=True)
file_out = os.path.join(dir_out, "sing_values.csv")
df_measurement.to_csv(file_out, sep="\t")


percent 200
[[-2.98023224e-08]
 [ 3.02679837e-09]
 [-4.05634637e-09]
 [-1.00044417e-10]
 [-5.45696821e-12]
 [ 1.00044417e-11]
 [ 1.54614099e-11]
 [ 9.09494702e-12]
 [-2.95585778e-12]
 [-1.13686838e-12]
 [-8.18545232e-12]
 [-3.97903932e-13]
 [ 9.09494702e-13]
 [-3.24007488e-12]
 [-1.98951966e-13]
 [ 8.18545232e-12]
 [-3.97903932e-13]
 [-2.55795385e-13]
 [-7.38964445e-13]
 [ 0.00000000e+00]
 [ 1.69109171e-12]
 [-6.39488462e-14]
 [ 2.13162821e-13]
 [-3.90798505e-13]
 [ 6.39488462e-14]
 [ 1.20792265e-13]
 [ 2.06057393e-13]
 [-2.77111667e-13]
 [ 2.13162821e-14]
 [-6.39488462e-13]
 [-2.55795385e-13]
 [-1.63424829e-13]
 [ 2.84217094e-14]
 [-4.26325641e-14]
 [ 0.00000000e+00]
 [-1.77635684e-13]
 [-2.91322522e-13]
 [ 2.98427949e-13]
 [ 2.84217094e-13]
 [ 0.00000000e+00]
 [-1.63424829e-13]
 [ 8.52651283e-14]
 [-3.55271368e-14]
 [ 1.42108547e-13]
 [ 7.10542736e-15]
 [-1.77635684e-13]
 [ 3.55271368e-14]
 [ 9.23705556e-14]
 [-2.27373675e-13]
 [ 7.88702437e-13]
 [ 7.38964445e-13]
 [-1.06581410e-13]


In [7]:
import matplotlib.pyplot as plt

plt.figure("name", figsize=(16, 8), dpi=80)
plt.xlabel("Percentage of the join size")
plt.ylabel("Relative error")

plt.title("Relative error in the computation of singular values using SVD-Figaro in comparison to SVD-MKL")

plt.yscale('log', base=10)
plt.locator_params(axis='x', nbins=6)
plt.locator_params(axis='x', nbins=10)
db_markers =  {"DBFavorita": "^", "DBYelp": "s", "DBRetailer": "x"}
db_colors = {"DBRetailer": "r", "DBFavorita": "g", "DBYelp": "y"}
comp_path = 
for exp_name in exp_names:
    df_measurement = df_measurement_exps[exp_name]
    for db_name in df_measurement:
        db_color = db_colors[db_name]
        db_marker = db_markers[db_name]
        plt.plot(df_measurement[db_name], "-" + db_color + db_marker, label="{} {}".format(exp_name, db_name))
plt.legend(loc="upper left")
plt.savefig('sing_val_accur.pdf')   # save the figure to file

plt.show()

SyntaxError: invalid syntax (3509863822.py, line 14)